In [509]:
import mysql.connector
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [351]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="password",
  database="twitter_db"
)

print(mydb)

In [352]:
N = 10000

In [353]:
tweet_list = []

cursor = mydb.cursor()

query = ("SELECT text FROM tweets WHERE lang='fr' LIMIT " + str(N))

cursor.execute(query)

for text in cursor:
    tweet_list.append(text[0])

cursor.close()

True

In [363]:
def hasNumbers(s):
    return any(c.isdigit() for c in s)

def clean_tweet(tweet):
    '''
        tokenize, lowercase, and lemmatize tweet
    '''
    stopwords = nltk.corpus.stopwords.words('french')
    other_stopwords = ['de', 'ou', 'sur', 'or', 'ni', 'car', 'quand', 'a', 'mais', 'donc', 'si', 'quel', 'entre',\
                       'tout', 'ce', 'cet', 'ça', 'tous', "c'est", 'faire', 'dans', 'fait']
    special_chars = ['+', '-', '%', '/', '#', '=', ':/']
    punctuation = [':', '.', '(', ')', ';', '?', '…', ',', '!', '...', '"']
    stopwords.extend(other_stopwords)
    
    tokenizer = TweetTokenizer()
    stemmer = SnowballStemmer('french')
    tokens = tokenizer.tokenize(tweet)
    
    tokens = tokens[:]
    # lowercase words
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    tokens = tokens[:]
    # stem words
    for i in range(len(tokens)):
        tokens[i] = stemmer.stem(tokens[i])

    # remove stopwords and punctuation
    clean_tokens =[]
    for token in tokens:
        token=token.replace('#','')
        if "'" in token:
            token = token.split("'",1)[1]
        if not (token in stopwords or(len(token)<=4 and "'" in token) or token in punctuation \
            or "@" in token or hasNumbers(token) or token in special_chars or "http" in token \
            or len(token) <= 2):
            clean_tokens.append(token)
            
    return clean_tokens

In [480]:
all_tokens = []

for i in range(N):    
    all_tokens.extend(clean_tweet(tweet_list[i]))

In [482]:
print(all_tokens)
print(len(all_tokens))

['arriv', 'caf', 'macron', 'alor', 'insoum', 'quoi', 'avril', 'invit', 'voisin', 'convainc', 'derni', 'indec', 'pourquoi', 'pass', 'extrait', 'interview', 'fillon', 'avant', 'pend', 'apres', 'primair', 'mdrrr', 'fin', 'dit', 'craint', 'degun', 'mec', 'doit', 'jlui', 'envoi', 'plein', 'photos', 'fillon', 'lach', 'veut', 'comprendr', 'rend', 'argent', 'fdp', 'couvert', 'antif', 'délinqu', 'extrêmegauch', 'holland', 'frontnational', 'marinelepen', 'new', 'infos', 'sarkozy', 'inspir', 'macron', 'depuis', 'temp', 'dis', 'progr', 'éco', 'macron', 'proch', 'celui', '. ..', 'asselineau', 'mouch', 'macron', 'ête', 'toujour', 'accord', 'mond', 'marin', 'pen', 'eclat', 'rir', 'macron', 'rit', 'jaun', 'macron', 'cultur', 'français', 'mélenchon', 'descendon', 'gaulois', 'hamon', 'follow', 'sondag', 'préfer', 'sondag', 'fillon', 'dupontaignan', 'droit', 'disent', 'côt', 'peupl', 'attaquent', 'seul', 'candidat', 'ressembl', 'peupl', 'comprendr', 'destruct', 'droit', 'travail', 'retour', 'notr', 'vidé

In [483]:
freq = nltk.FreqDist(all_tokens) 

sorted_tokens = sorted(freq.items(), key=lambda kv: kv[1], reverse=True)

sorted_tokens

In [486]:
len(sorted_tokens)

6573

In [487]:
sorted_tokens = np.array(sorted_tokens)

In [488]:
tokens = sorted_tokens[:5000,0]

In [531]:
def vectorize_tweets(tweets, tokens):
    '''
        tweets : list of tweets
        tokens : vector features
        clean and vectorize tweets
    '''
    vectorized_tweets = []
    corpus = []
    for tweet in tweets:
        tweet = clean_tweet(tweet)
        '''
        processed_tweet = []
        for word in tweet:
            if word in tokens:
                processed_tweet.append(word)
        '''
        corpus.append(' '.join(tweet))
    vectorizer = CountVectorizer(vocabulary=tokens)
    X = vectorizer.fit_transform(corpus)
    return X

In [546]:
X = vectorize_tweets(tweet_list, tokens).toarray()

In [547]:
np.where(X[0] == 1)

(array([  1,  60, 101]),)

In [549]:
X.shape

(10000, 5000)

In [558]:
random_tweets = X[np.random.choice(N, 2, replace=False)]

In [554]:
def cos_sim(a,b):
    dot_product = np.dot(a,b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [557]:
cos_sim(random_tweets[0], random_tweets[1])

0.0